In [ ]:
# AutoEncoders

"""**************************************************************************************************************************"""

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing the dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# Preparing the training set and the test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

"""
# 1). The whole process given can be done by pivot_table in DataFrame also.
# 2). First in the above process do not convert DataFrame into array.
# 3). Then use code given below:-

     training_set.columns = ['user_id', 'movie_id', 'rating', 'qwe']
     pv = pd.pivot_table(data=training_set, values='rating', index='user_id', columns='movie_id' )
     test_set.columns = ['user_id', 'movie_id', 'rating', 'qwe']
     pv2 = pd.pivot_table(data=test_set, values='rating', index='user_id', columns='movie_id' )
    for var in range(1592, 1683):
         pv2[var] = np.NaN
		 
# 4). Now convert all NaN values into zero and then convert DataFrame into array. 

"""
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

"""**************************************************************************************************************************"""

# Till now the code of Auto-Encoder and Boltzmann machine are same.

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


# Creating the architecture of the Neural Network
class SAE(nn.Module):    # Inheriting the properties of nn.Module 
    # SAE = stacked Auto Encoder.
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
					# 1a). Here fc1 is a full connection. It is a connection of input layer and 1st hidden layer.
					# 1b) Number of nodes in input layer are nb_movies. 
		            # 1). Here we are using no_movies.
					# 2). Here 20 is number of nodes in hidden layer. Experiment it and check accuracy but start with this 20.
					   
        self.fc2 = nn.Linear(20, 10)
		          # 1). Here fc2 is a full connection. Here we are connecting 1st hidden layer with 2nd hidden layer.
				  # 2). 20 = number of nodes in 1st hidden layer.
				  # 3). 10 = number of nodes in 2nd hidden layer.
				  
        self.fc3 = nn.Linear(10, 20)
		          # 1). Here fc3 is a full connection. Here we are connecting 2nd hidden layer with 3rd hidden layer.
				  # 2). 10 = number of nodes in 2nd hidden layer.
				  # 3). 20 = number of nodes in 3rd hidden layer.
				  
        self.fc4 = nn.Linear(20, nb_movies)
		          # 1). Here we are using no_movies
				  # 1a). Here fc4 is a full connection. Here we are connecting 3rd hidden layer with visible output layer.
				  # 2). 20 = number of nodes in 3rd hidden layer.
				  # 3). nb_movies = number of nodes in visible output layer.
				  
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
print(criterion)
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)
print(optimizer)

"""**************************************************************************************************************************"""

# Training the SAE
nb_epoch = 200    # nb_epoch = number of round.
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()      # Target vector, this is same as of input matrix. Creating this to find out error.
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

"""**************************************************************************************************************************"""

# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)     # Here we are using training set.
    target = Variable(test_set[id_user])                   # Here we are using test set.
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

"""**************************************************************************************************************************"""
# PREDICTION ON NEW INPUT:-
def predict( self, x): # x: visible nodes
  _, h = self.sample_h( x)
  _, v = self.sample_v( h)
  return v
  
"""**************************************************************************************************************************"""




  





